In [1]:
import pandas as pd
import os
import wget
import ssl
import fitz
import ocrmypdf

# This restores the same behavior as before.
#context = ssl._create_unverified_context()

ssl._create_default_https_context = ssl._create_unverified_context


In [2]:
#Base completa
latam = pd.read_excel('TodoLatam2.xlsx')
#filtro columnas de interés
latam = latam.loc[:,['País', 'Entidad','Tipo convocatoria', 'Título', 'Links pdf', 'Link']]
#limpieza de categorías
latam.replace({'Tipo convocatoria': 'Índices/Evaluación'}, {'Tipo convocatoria': 'Índices-Evaluación'}, regex=True, inplace=True)
latam.replace({'Tipo convocatoria': 'Investigación/Innovación'}, {'Tipo convocatoria': 'Investigación-Innovación'}, regex=True, inplace=True)

######################
## Paises
#paises = latam['País'].unique().tolist()
#for pais in paises:
pais = 'Colombia'
pais_select = latam.loc[latam['País']==pais]
pais_select = pais_select[0:10]
#tipo convocatorial
tipo_convocatoria = pais_select['Tipo convocatoria'].unique().tolist()
#for convocarioria in tipo_convocatoria:
entidad = pais_select['Entidad'].unique().tolist()


In [8]:
### Crea Carpeta País

directoria_base = '/home/duban/Workspace/Analisis-Scrapping-Convocatorias-Clacso/test_descarga'
pais = 'Colombia'

os.mkdir(pais)

###################
#Crea carpeta de entidades


base_pais = '/home/duban/Workspace/Analisis-Scrapping-Convocatorias-Clacso/test_descarga/' + pais


entidades = []
for ent in entidad:
        #Crea carpeta de estado del proyecto
    try:
        base_entidad = base_pais+ '/' + ent
        entidades.append(base_entidad)
        os.mkdir(base_entidad)
    except FileExistsError:
        continue 

############################
#Crea carpeta por tipo de proyecto
tipo_convocatorias= []
#bases_proyectos = []
#Entidades
for ent in entidades:
    #Convocatorias
    for convocatoria in tipo_convocatoria:
        
        #Crea carpeta de estado del proyecto
        try:
            base_tipo_convocatoria = ent+ '/' + convocatoria
            tipo_convocatorias.append(base_tipo_convocatoria)
            os.mkdir(base_tipo_convocatoria)

            #####
            #Crea carpeta por proyecto
            tipo_proyecto = pais_select.loc[pais_select['Tipo convocatoria']==convocatoria]
            #Proyectos

            
            for count, proyecto in enumerate(tipo_proyecto['Título']):
                
                base_proyecto = base_tipo_convocatoria+ '/' + str(count+1) + '.' + proyecto[0:150]
                os.mkdir(base_proyecto)
                 # Descarga archivos
                proyecto_pdf = tipo_proyecto.loc[tipo_proyecto['Título']==proyecto]
                
                n_proyecto = 0            
                for count_proyecto, alist_link in enumerate(proyecto_pdf['Links pdf']):
                    
                    pdfs_brutos = alist_link.split(', ')
                     # Selecciona solo pdfs
                    pdfs = [pdf for pdf in pdfs_brutos if pdf.endswith('.pdf')]

                    texto_proyecto = ''
                    #n_proyecto = 0 

                    #Recorre los pdfs
                    for count_link, pdf in enumerate(pdfs):
                        path = base_proyecto  + '/' + str(count_link+1) + '.' + pdf.split('/')[-1]
                        wget.download(pdf, path)
                        ### Ejecuta OCR
                        try:
                            ocrmypdf.ocr(path, path, deskew=True)
                        except:
                            doc = fitz.open(path)

                        doc = fitz.open(path)

                        #Pagina por pagina y extrae txto
                        
                        for pagina in doc:
                            text = pagina.getText()#.encode('utf8')
                            texto_proyecto = texto_proyecto + str(text) + ' '
                        texto_proyecto = texto_proyecto.strip(' ')
                        #doc.close()

                    nom_txt = 'text' + str(count_proyecto) + '.txt'
                    #n_proyecto+=1
                    print(nom_txt)
                    print(count_proyecto)   
                #txt = open(nom_txt, 'w')
                #txt.write(str(texto_proyecto))
                #txt.close()
                #n_proyecto+=1

        except FileExistsError:
            continue

Scanning contents: 100%|██████████| 4/4 [00:00<00:00, 45.97page/s]
OCR:   0%|          | 0.0/4.0 [00:00<?, ?page/s]
Scanning contents: 100%|██████████| 22/22 [00:00<00:00, 158.72page/s]
This PDF has a fillable form. Chances are it is a pure digital document that does not need OCR.
OCR:   0%|          | 0.0/22.0 [00:00<?, ?page/s]
Scanning contents: 100%|██████████| 22/22 [00:00<00:00, 182.14page/s]
OCR:   0%|          | 0.0/22.0 [00:00<?, ?page/s]
Scanning contents: 100%|██████████| 275/275 [00:01<00:00, 170.85page/s]
OCR:   0%|          | 0.0/275.0 [00:00<?, ?page/s]
Scanning contents: 100%|██████████| 34/34 [00:00<00:00, 173.44page/s]
OCR:   0%|          | 0.0/34.0 [00:00<?, ?page/s]
text0.txt
0
Scanning contents: 100%|██████████| 3/3 [00:00<00:00, 57.87page/s]
OCR:   0%|          | 0.0/3.0 [00:00<?, ?page/s]
Scanning contents: 100%|██████████| 19/19 [00:00<00:00, 44.14page/s]
OCR:   0%|          | 0.0/19.0 [00:00<?, ?page/s]
Scanning contents: 100%|██████████| 3/3 [00:00<00:00, 50.5

KeyboardInterrupt: 

In [ ]:
count_proyecto+1

In [ ]:

n_proyecto+=1
n_proyecto

In [ ]:
### Descarga pdfs y convierte en txt

texto_proyectos = []
 
base_proyecto = '/home/duban/Workspace/Analisis-Scrapping-Convocatorias-Clacso/test_descarga/Colombia/'
n = 0
for count_proyecto, alist_link in enumerate(colombia['Links pdf'][0:1]):
    pdfs = []
    pdfs_brutos = alist_link.split(', ')
    for p in pdfs_brutos:
        if p.endswith('.pdf'):
            pdfs.append(p)
     #print(count_proyecto+1, pdfs)
    texto_proyecto = ''

    for count_link, pdf in enumerate(pdfs):
         #print(str(count_proyecto+1) + '.' + str(count_link+1), pdf)
        path = base_proyecto + proyectos[n] + '/' + str(count_proyecto+1) + '.' + str(count_link+1) + '.' + pdf.split('/')[-1]
        wget.download(pdf, path)
         
         ### Extrae Texto
        try:
            ocrmypdf.ocr(path, path, deskew=True)
        except:
             doc = fitz.open(path)

        doc = fitz.open(path)
         
        for pagina in doc:
            text = pagina.getText()#.encode('utf8')
            texto_proyecto = texto_proyecto + str(text) + ' '
        texto_proyecto = texto_proyecto.strip(' ')
     
    #texto_proyectos.append(texto_proyecto)
    txt = open('.txt', 'w')
    txt.write(str(texto_proyectos[0]))
    txt.close()
    n+=1
#salida.close()

In [ ]:
f = open('.txt', 'w')
f.write(str(texto_proyectos[0]))
f.close()

In [ ]:
texto_proyectos

In [ ]:
colombia['Links pdf'][0:1][0]